## Prediction

In [1]:
import os

In [2]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection/notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection'

In [1]:
import tensorflow as tf
from tensorflow import keras
from plantDiseaseDetection.config import ConfigurationManager
from plantDiseaseDetection.constants import *
from plantDiseaseDetection.utils import read_yaml
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

2023-11-15 11:53:50.967474: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 11:53:51.004791: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 11:53:51.004831: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 11:53:51.004861: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 11:53:51.012428: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 11:53:51.013012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
class PlantDisease:

    def __init__(self, filename):
        self.filename = filename
        self.config = ConfigurationManager()
        self.params = read_yaml(PARAMS_FILE_PATH)
        self.training_config = self.config.get_training_config()
        self.data_ingestion_config = self.config.get_data_ingestion_config()

    def _get_prediction_classes(self):
        classes = sorted(os.listdir(self.data_ingestion_config.dataset_dir))
        class_id_to_label = {i: class_name for i, class_name in enumerate(classes)}
        return class_id_to_label
    
    def predict(self):
        
        # Load Model
        model = load_model(self.training_config.trained_model_path)

        img_name = self.filename
        target_img_size = tuple(self.params.IMAGE_SIZE[:2])
        test_image = load_img(img_name, target_size = target_img_size)
        
        test_image = img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        predictions = model.predict(test_image)
        result = np.argmax(predictions)

        prediction_classes = self._get_prediction_classes()
        predicted_class_label = prediction_classes[result]
        
        return [{'image': predicted_class_label}]